In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import pickle

In [2]:
#first we need to read in our Government Responses Dataset
govResponse = pd.read_excel("Gov_Responses.xlsx", sheet_name = "Dataset")
govResponse

,country,geoid,iso,d,cases,deaths,school,school_local,domestic,domestic_local,...,wage,credit,taxc,taxd,export,rate,Rigidity_Public_Health,Economic_Measures,population_2019,continent
0,Aruba,AW,ABW,2020-01-01 00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,106310.0,America
1,Aruba,AW,ABW,2020-01-02 00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,106310.0,America
2,Aruba,AW,ABW,2020-01-03 00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,106310.0,America
3,Aruba,AW,ABW,2020-01-04 00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,106310.0,America
4,Aruba,AW,ABW,2020-01-05 00:00:00,0.0,0.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,106310.0,America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62695,Hong Kong,HK,HKG,2020-09-29 00:00:00,NaN,NaN,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.461539,0.714286,NaN,NaN
62696,Hong Kong,HK,HKG,2020-09-30 00:00:00,NaN,NaN,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.461539,0.714286,NaN,NaN
62697,Macau,MO,MAC,2020-09-30 00:00:00,NaN,NaN,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.250000,0.714286,NaN,NaN
62698,Hong Kong,HK,HKG,2020-10-01 00:00:00,NaN,NaN,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.461539,0.714286,NaN,NaN


In [3]:
#next we need to read in our better covid dataset
df = pd.read_excel("covid-data.xlsx")
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511


In [4]:
#Now let us read in our dataset from which we will extract the countries
spread_chain = pd.read_csv("Spread_Chain.csv")
spread_chain = spread_chain.dropna()
spread_chain.head()

,Source Country,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vatican City,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
0,Italy,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,China,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Iran,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,United Kingdom,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,France,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [60]:
#now lets get our list of countries
countries = spread_chain.columns.tolist()
countries.append("United States of America")
print(len(countries))
countries

188


['Source Country',
 'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Ha

In [51]:
#ok now lets get rid of any country not in our list that is in the government response dataset
govResponse = govResponse[govResponse["country"].isin(countries)]
govResponse

,country,geoid,iso,d,cases,deaths,school,school_local,domestic,domestic_local,...,wage,credit,taxc,taxd,export,rate,Rigidity_Public_Health,Economic_Measures,population_2019,continent
275,Afghanistan,AF,AFG,2020-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,38000000.0,Asia
276,Afghanistan,AF,AFG,2020-01-02 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,38000000.0,Asia
277,Afghanistan,AF,AFG,2020-01-03 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,38000000.0,Asia
278,Afghanistan,AF,AFG,2020-01-04 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,38000000.0,Asia
279,Afghanistan,AF,AFG,2020-01-05 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,38000000.0,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61320,Taiwan,TW,TWN,2020-09-27 00:00:00,0.0,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,1.0,0.0,0.0,NaN,0.285714,24000000.0,Asia
61321,Taiwan,TW,TWN,2020-09-28 00:00:00,0.0,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,1.0,0.0,0.0,NaN,0.285714,24000000.0,Asia
61322,Taiwan,TW,TWN,2020-09-29 00:00:00,3.0,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,1.0,0.0,0.0,NaN,0.285714,24000000.0,Asia
61323,Taiwan,TW,TWN,2020-09-30 00:00:00,1.0,0.0,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,1.0,0.0,0.0,NaN,0.285714,24000000.0,Asia


In [52]:
len(govResponse.country.unique())

176

In [9]:
#ok now lets do the same thing for our general Covid 
covidData = df[df["location"].isin(countries)]
covidData

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80981,ZWE,Africa,Zimbabwe,2021-04-07,36984.0,18.0,14.571,1531.0,0.0,1.143,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
80982,ZWE,Africa,Zimbabwe,2021-04-08,37052.0,68.0,22.286,1532.0,1.0,1.286,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
80983,ZWE,Africa,Zimbabwe,2021-04-09,37147.0,95.0,34.857,1535.0,3.0,1.571,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571
80984,ZWE,Africa,Zimbabwe,2021-04-10,37273.0,126.0,51.714,1538.0,3.0,2.000,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.571


In [10]:
#ok now we got the countries right! the COVID-19 data goes farther than our government regulation data
#so lets save it as a new dataset just in case we need it the future
covidData.to_excel("covidDataUpdated.xlsx", index = False)

In [53]:
#we also don't need a lot of this data, we can get rid of some of the columns
#now it is tempting to use a lot of these regulations, but that creates a difficult time establishing baselines
#mainly we want travel, testing and masks
regulations = ['travel',
 'travel_partial',
 'travel_dom',
 'travel_dom_partial',
 'testing',
 'testing_narrow',
 'masks']

In [54]:
#Ok now lets get some statistics first we need a baseline for when a country will implement any given 
#government regulation, for the first time, and then any subsequent times.
#we can set this up in a dictionary of dictionaries, first lets create our overeaching dictionary
countryRestrictions = {}
#now lets iterate over our government response data to start getting informtion
#we need to set the starting country as afghanistan so we can start adding information, plus add the starting info
currCountry = "Afghanistan"
totalInfect = 0
totalDeath = 0
currDict = {}
infected = False
currentInfect = 0
reset = []

for index, row in govResponse.iterrows():
    #lets check if our country is not our dictionary
    if row['country'] != currCountry:
        print("Starting on:",row['country'])

        countryRestrictions[currCountry] = currDict
        #Here is a good place to put our progress message
        #print("Just finished country:", row['country'], " Overall regulations were:", currDict.keys())
        #set our current country to the new country
        currCountry = row['country']
        #set our current dict to an empty dictionary
        currDict = {}
        #we also want to set our infected flag back to false since we are moving onto a new country
        infected = False
        #and lets also set our total deaths and infections both to 0
        totalInfect = 0
        totalDeath = 0
        #we also want a currentinfected tracker, and a timer to reset that tracker, covid is 10 days
        currentInfect = 0
        reset = []
    #now lets check the to see if we are already infected, if so lets start keeping track of numbers
    if infected:
        #ok so we are infected lets set our current regulations, and add today's data
        totalInfect += row['cases']
        totalDeath += row['deaths']
        reset.append(row['cases'])
        currentInfect += row['cases']
        if len(reset) > 10:
            currentInfect -= reset[0]
            del reset[0]
        #we want a list in string format of the current regulations as the dictionary key
        currentReg = ""
        for regulation in regulations:
            if row[regulation] == 1:
                currentReg += regulation + ","
        
        #lets check if we have no regulations, if so we need to make it our baseline for infection
        if currentReg == "":
            currentReg = "baseline"
        
        #now lets add these to the dictionary under the appriopriate regulation
        if currentReg in currDict:
            currDict[currentReg].append([row['cases'], row['deaths'], currentInfect, totalInfect, totalDeath])
        else:
            currDict[currentReg] = [[row['cases'], row['deaths'], currentInfect, totalInfect, totalDeath]]
    else:
        #ok we are not infected lets check if we are infected and update the flag if so
        if row['cases'] > 0:
            infected = True
        

Starting on: Angola
Starting on: Albania
Starting on: Andorra
Starting on: United Arab Emirates
Starting on: Argentina
Starting on: Armenia
Starting on: Antigua and Barbuda
Starting on: Australia
Starting on: Austria
Starting on: Azerbaijan
Starting on: Burundi
Starting on: Belgium
Starting on: Benin
Starting on: Burkina Faso
Starting on: Bangladesh
Starting on: Bulgaria
Starting on: Bahrain
Starting on: Bahamas
Starting on: Bosnia and Herzegovina
Starting on: Belarus
Starting on: Belize
Starting on: Bolivia
Starting on: Brazil
Starting on: Barbados
Starting on: Bhutan
Starting on: Botswana
Starting on: Canada
Starting on: Switzerland
Starting on: Chile
Starting on: China
Starting on: Cote d'Ivoire
Starting on: Cameroon
Starting on: Colombia
Starting on: Comoros
Starting on: Cabo Verde
Starting on: Costa Rica
Starting on: Cuba
Starting on: Cyprus
Starting on: Germany
Starting on: Djibouti
Starting on: Dominica
Starting on: Denmark
Starting on: Dominican Republic
Starting on: Algeria
St

In [55]:
print(countryRestrictions['United States of America'])

{'baseline': [[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 1.0, 1.0, 0.0], [0.0, 0.0, 1.0, 1.0, 0.0], [3.0, 0.0, 4.0, 4.0, 0.0], [0.0, 0.0, 4.0, 4.0, 0.0], [0.0, 0.0, 4.0, 4.0, 0.0], [0.0, 0.0, 4.0, 4.0, 0.0], [1.0, 0.0, 5.0, 5.0, 0.0], [1.0, 0.0, 6.0, 6.0, 0.0], [1.0, 0.0, 7.0, 7.0, 0.0], [3.0, 0.0, 10.0, 10.0, 0.0], [0.0, 0.0, 9.0, 10.0, 0.0], [0.0, 0.0, 9.0, 10.0, 0.0], [1.0, 0.0, 7.0, 11.0, 0.0], [0.0, 0.0, 7.0, 11.0, 0.0], [0.0, 0.0, 7.0, 11.0, 0.0], [0.0, 0.0, 7.0, 11.0, 0.0], [0.0, 0.0, 6.0, 11.0, 0.0], [1.0, 0.0, 6.0, 12.0, 0.0], [0.0, 0.0, 5.0, 12.0, 0.0], [1.0, 0.0, 3.0, 13.0, 0.0], [1.0, 0.0, 4.0, 14.0, 0.0], [0.0, 0.0, 4.0, 14.0, 0.0], [0.0, 0.0, 3.0, 14.0, 0.0], [0.0, 0.0, 3.0, 14.0, 0.0], [0.0, 0.0, 3.0, 14.0, 0.0], [0.0, 0.0, 3.0, 14.0, 0.0], [0.0, 0.0, 3.0, 14.0, 0.0], [1.0, 0.0, 3.0, 15.0, 0.0], [19.0, 0.0, 22.0, 34.0, 0.0], [0.0, 0.0, 21.0, 34.0, 0.0], [0.0, 0.0, 20.0, 34.0, 0.0], [18.0, 0.0, 38.0, 52.0, 0.0], [0.0, 0.0, 

In [56]:
#now lets save this dictionary
with open('countryRestrictions.pickle', 'wb') as handle:
    pickle.dump(countryRestrictions, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [58]:
touristsInc = pd.read_excel("touristsIncome.xlsx")
touristsOut = pd.read_excel("touristsOutgo.xlsx")
print(touristsInc.head())
print(touristsOut.head())

       Country  Number of tourists
0        Aruba           1951000.0
1  Afghanistan                 NaN
2       Angola            218000.0
3      Albania           6406000.0
4      Andorra           8235000.0
       Country   tourists
0        Aruba        NaN
1  Afghanistan        NaN
2       Angola        NaN
3      Albania  5922000.0
4      Andorra        NaN


In [61]:
#now lets get tourist information by country
inbounds = {}
outbounds = {}
for index, row in touristsInc.iterrows():
    if row["Country"] in countries and row["Number of tourists"] > 0:
        inbounds[row["Country"]] = row["Number of tourists"]

for index, row in touristsOut.iterrows():
    if row["Country"] in countries and row["tourists"] > 0:
        outbounds[row["Country"]] = row["tourists"]
    
print("inbounds is: ", inbounds)
print("outbounds is: ", outbounds)

inbounds is:  {'Angola': 218000.0, 'Albania': 6406000.0, 'Andorra': 8235000.0, 'United Arab Emirates': 21553000.0, 'Argentina': 7399000.0, 'Armenia': 1894000.0, 'Antigua and Barbuda': 1035000.0, 'Australia': 9466000.0, 'Austria': 31884000.0, 'Azerbaijan': 3170000.0, 'Belgium': 9343000.0, 'Benin': 337000.0, 'Burkina Faso': 143000.0, 'Bangladesh': 323000.0, 'Bulgaria': 12552000.0, 'Bahrain': 11061000.0, 'Bosnia and Herzegovina': 1198000.0, 'Belarus': 11832000.0, 'Belize': 1674000.0, 'Bolivia': 1239000.0, 'Brazil': 6353000.0, 'Barbados': 966000.0, 'Bhutan': 316000.0, 'Canada': 32430000.0, 'Switzerland': 11818000.0, 'Chile': 5431000.0, 'China': 162538000.0, "Cote d'Ivoire": 2070000.0, 'Colombia': 4529000.0, 'Comoros': 45099.9984741211, 'Cabo Verde': 758000.0, 'Costa Rica': 3366000.0, 'Cuba': 4276000.0, 'Cyprus': 4117000.0, 'Germany': 39563000.0, 'Dominica': 322000.0, 'Denmark': 32903000.0, 'Dominican Republic': 7550000.0, 'Algeria': 2371000.0, 'Ecuador': 2108000.0, 'Spain': 126170000.0, 'E

In [69]:
#Now here comes the heavy lifting, we want to figure out roughly where all the tourists go
#we will do this by taking all outbounds tourist countries and splitting there tourists, by percentage wise 
#first we need to calculate the percentage of inbound tourists every country has
#sum up the total number of inbound tourists
inboundsSum = sum(inbounds.values())
inboundsPrec = {}
for key, value in inbounds.items():
    inboundsPrec[key] = value/inboundsSum

#now for everyone outbounds country we need a dictionary stating roughly how many people will travel to there.
#More tourists will travel to closer regions but we can leave that alone for now.
allTourists = {}
for key1, value1 in outbounds.items():
    currCountry = {}
    for key2, value2 in inboundsPrec.items():
        currCountry[key2] = value2*value1
    allTourists[key1] = currCountry


19061000.000000004


In [62]:
#Now we need to group the countries into regions
my_file = open("countries-region.txt", "r")
reg = my_file.read()

reg = reg.split("\n")

regions = {}
currRegion = ""

for region in reg:
    if region and region[0] == "[":
        currRegion = region
    else:
        if region in countries:
            regions[region] = currRegion

regions

{'Australia': '[ Australia and New Zealand ]',
 'New Zealand': '[ Australia and New Zealand ]',
 'Antigua and Barbuda': '[ Caribbean ]',
 'Bahamas': '[ Caribbean ]',
 'Barbados': '[ Caribbean ]',
 'Cuba': '[ Caribbean ]',
 'Dominica': '[ Caribbean ]',
 'Dominican Republic': '[ Caribbean ]',
 'Grenada': '[ Caribbean ]',
 'Haiti': '[ Caribbean ]',
 'Jamaica': '[ Caribbean ]',
 'Saint Kitts and Nevis': '[ Caribbean ]',
 'Saint Lucia': '[ Caribbean ]',
 'Saint Vincent and the Grenadines': '[ Caribbean ]',
 'Trinidad and Tobago': '[ Caribbean ]',
 'Belize': '[ Central America ]',
 'Costa Rica': '[ Central America ]',
 'El Salvador': '[ Central America ]',
 'Guatemala': '[ Central America ]',
 'Honduras': '[ Central America ]',
 'Mexico': '[ Central America ]',
 'Nicaragua': '[ Central America ]',
 'Panama': '[ Central America ]',
 'Kazakhstan': '[ Commonwealth of Independent States ]',
 'Kyrgyzstan': '[ Commonwealth of Independent States ]',
 'Tajikistan': '[ Commonwealth of Independent Sta

In [63]:
#Now lets do land borders
borders = pd.read_csv("borders.csv")
borders.head()

,country_code,country_name,country_border_code,country_border_name
0,AD,Andorra,FR,France
1,AD,Andorra,ES,Spain
2,AE,United Arab Emirates,OM,Oman
3,AE,United Arab Emirates,SA,Saudi Arabia
4,AF,Afghanistan,CN,China


In [64]:
#we need to get land borders for every country
landBorders = {}
for index, row in borders.iterrows():
    if row["country_name"] in countries and row["country_border_name"] in countries:
        if row["country_name"] in landBorders:
            landBorders[row["country_name"]].append(row["country_border_name"])
        else:
            landBorders[row["country_name"]] = [row["country_border_name"]]

landBorders

{'Andorra': ['France', 'Spain'],
 'United Arab Emirates': ['Oman', 'Saudi Arabia'],
 'Afghanistan': ['China', 'Pakistan', 'Tajikistan', 'Uzbekistan'],
 'Albania': ['Greece', 'Montenegro', 'North Macedonia', 'Serbia'],
 'Armenia': ['Azerbaijan', 'Georgia', 'Turkey'],
 'Angola': ['Namibia', 'Zambia'],
 'Argentina': ['Brazil', 'Chile', 'Paraguay', 'Uruguay'],
 'Austria': ['Germany',
  'Hungary',
  'Italy',
  'Liechtenstein',
  'Slovakia',
  'Slovenia',
  'Switzerland'],
 'Azerbaijan': ['Armenia', 'Georgia', 'Turkey'],
 'Bosnia and Herzegovina': ['Croatia', 'Montenegro', 'Serbia'],
 'Bangladesh': ['India', 'Myanmar'],
 'Belgium': ['France', 'Germany', 'Luxembourg', 'Netherlands'],
 'Burkina Faso': ['Benin', 'Ghana', 'Mali', 'Niger', 'Togo'],
 'Bulgaria': ['Greece', 'North Macedonia', 'Romania', 'Serbia', 'Turkey'],
 'Burundi': ['Rwanda'],
 'Benin': ['Burkina Faso', 'Niger', 'Nigeria', 'Togo'],
 'Brazil': ['Argentina',
  'Colombia',
  'Guyana',
  'Paraguay',
  'Peru',
  'Suriname',
  'Urugu

In [70]:
#Ok now lets pickle all of our files!
with open('touristNumbers.pickle', 'wb') as handle:
    pickle.dump(allTourists, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('landBorders.pickle', 'wb') as handle:
    pickle.dump(landBorders, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('countryRegions.pickle', 'wb') as handle:
    pickle.dump(regions, handle, protocol=pickle.HIGHEST_PROTOCOL)